In [ ]:
%load_ext autoreload

In [ ]:
!pip install pytest pytest-bdd

In [ ]:
from lavague.core import  WorldModel, ActionEngine
from lavague.core.agents import WebAgent
from lavague.drivers.selenium import SeleniumDriver

In [ ]:
# llm = gemini...
# pass llm to action engine

selenium_driver = SeleniumDriver(headless=False)
world_model = WorldModel()
action_engine = ActionEngine(selenium_driver)
agent = WebAgent(world_model, action_engine)

In [ ]:

test_case = """
Scenario: Submission with missing phone number
      Given I am on the job application page
      When I enter "John" in the "First Name" field
      And I enter "Doe" in the "Last Name" field
      And I enter "john.doe@example.com" in the "Email Address" field
      And I leave the "Phone Number" field empty
      And I enter "I am very interested in this position." in the "Cover Letter" field
      And I click the "Apply" button
      Then I should see an error message for the "Phone Number" field
"""

URL = "https://form.jotform.com/241472287797370"

OBJECTIVE = f"Run this test case: \n\n{test_case}"



In [ ]:
agent.get(URL)
agent.run(OBJECTIVE)

In [ ]:
import pandas as pd

logs = agent.logger.return_pandas()

# logs = pd.read_csv("logs_29052023-1157.csv")

In [ ]:
logs.head()

In [ ]:
logs.info()

In [ ]:
# extract full code

full_code = "\n".join(logs['code'].dropna())
full_code

In [ ]:
last_step = logs.iloc[-1]

In [ ]:
# handle last screenshot

# # extract last screenshot of last step (test outcome)
# last_screenshot = last_step['screenshots'][-1]
# last_screenshot.show()
# # last_screenshot.path()


from PIL import Image
import base64
from io import BytesIO
import os

def get_latest_screenshot_path(directory):
    # List all files in the directory
    files = os.listdir(directory)
    
    # Get the full path of the files
    full_paths = [os.path.join(directory, f) for f in files]
    
    # Find the most recently modified file
    latest_file = max(full_paths, key=os.path.getmtime)
    
    return latest_file

def pil_image_to_base64(image_path):
    # Open the image file
    with Image.open(image_path) as img:
        # Convert image to BytesIO object
        # img.show()
        buffered = BytesIO()
        img.save(buffered, format="PNG")
        # Encode the BytesIO object to base64
        img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str

last_screenshot_path = get_latest_screenshot_path(last_step["screenshots_path"])
b64_img = pil_image_to_base64(last_screenshot_path)

In [ ]:
last_html = last_step['html']
last_html

In [ ]:
nodes = action_engine.get_nodes(f"We have ran the test case, generate a pytest-bdd assert statement.\n\ntest case:\n{test_case}")

In [ ]:
nodes

In [ ]:
EXAMPLES = """
import pytest
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pytest_bdd import scenarios, given, when, then

# Constants
BASE_URL = 'https://form.jotform.com/241472287797370'

# Scenarios
scenarios('test_form_submission.feature')

# Fixtures
@pytest.fixture
def browser():
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    driver.get(BASE_URL)
    yield driver
    driver.quit()

# Steps
@given('I am on the job application page')
def i_am_on_the_job_application_page(browser):
    pass

@when('I enter "John" in the "First Name" field')
def i_enter_first_name(browser):
    first_name_field = browser.find_element(By.XPATH, "/html/body/form/div[1]/ul/li[2]/div/div/span[1]/input")
    first_name_field.send_keys("John")

@when('I enter "Doe" in the "Last Name" field')
def i_enter_last_name(browser):
    last_name_field = browser.find_element(By.XPATH, "/html/body/form/div[1]/ul/li[2]/div/div/span[2]/input")
    last_name_field.send_keys("Doe")

@when('I enter "john.doe@example.com" in the "Email Address" field')
def i_enter_email_address(browser):
    email_field = browser.find_element(By.XPATH, "/html/body/form/div[1]/ul/li[3]/div/span/input")
    email_field.send_keys("john.doe@example.com")

@when('I enter "(123) 456-7890" in the "Phone Number" field')
def i_enter_phone_number(browser):
    phone_number_field = browser.find_element(By.XPATH, "/html/body/form/div[1]/ul/li[4]/div/span/input")
    phone_number_field.send_keys("(123) 456-7890")

@when('I leave the "Cover Letter" field empty')
def i_leave_cover_letter_empty():
    # No action needed as the field should remain empty
    pass

@when('I click the "Apply" button')
def i_click_apply_button(browser):
    apply_button = WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/form/div[1]/ul/li[6]/div/div/button"))
    )
    browser.execute_script("arguments[0].scrollIntoView(true);", apply_button)
    apply_button.click()

@then('I should see an error message for the "Cover Letter" field')
def i_should_see_error_message(browser):
    try:
        error_message = browser.find_element(By.XPATH, "/html/body/form/div[1]/ul/li[5]/div/div/span")
        assert error_message.is_displayed()
    except Exception as e:
        pytest.fail(f"Error message not displayed: {e}")
"""
SYSTEM_PROMPT = "You are an expert in software testing frameworks and python code generation. Your task is to read Gherkin tests, Selenium code, HTML and Screenshots of pages to generate assert statements as well as package everything into python-bdd code. Use descriptive function names, use execute_script if you think a click could result in an ElementClickInterceptedException, always use try-except blocks to catch exceptions and raise pytest.fail when checking the assert condition step when needed, name the scenario accordingly, follow good practices. Only output the code."
USER_PROMPT = f"""
We have ran all navigation steps of the test case, generate an assert statement and a valid pytest-bdd code (has to contain fixtures, scenario, gherkin style definitions, etc) containing every steps and the assert with the following inputs:

Base url:{URL}\n
Test case:{test_case}\n
Already executed code:\n{full_code}\n
selected html of the last page:{nodes}

here are some examples of expected code:\n\n{EXAMPLES}
"""


In [ ]:
# convert the selenium code into a pytest-bdd file
from openai import OpenAI
def generate_pytest_file():
    print(test_case)
    api_key = ""
    client = OpenAI(api_key=api_key)
    # Define the code modification prompt
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system", 
                "content": SYSTEM_PROMPT
            },
            {
                "role": "user", 
                "content": [
                    {"type": "text", "text": USER_PROMPT},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{b64_img}"
}}
                ]
            }
        ]
    )

    return completion.choices[0].message.content


In [ ]:
# instruction = f"The following test case has been executed, write an assert statement to verify the outcome of the test case.\n\n{test_case}"
# nodes = action_engine.get_nodes(instruction)

content = generate_pytest_file()

In [ ]:
print(content)

In [ ]:
pytest_code = content.strip("```python")
pytest_code = pytest_code.strip("```")
pytest_code = pytest_code.strip("```\n")

In [ ]:
print(pytest_code)

In [ ]:
with open("tests/test_case.py", "w") as f:
    f.write(pytest_code)

In [ ]:
!pytest -v tests